### King County Request | December 2022

"Total onroad vehicle emissions and activity data within each jurisdiction's geographic boundary for the calendar years 2019 and 2020. If available, we would also request projections over time going to 2050. Only aggregate data is needed, broken down by the vehicle mode and fuel type, as shown in the table. If there is data that doesn't fit into the listed categories, please let us know."

Requested for all King County cities, King County total and King County unicorporated areas.

Years required include 2018, 2019, 2020, 2030, 2040, 2050. Interpolation between 2018 and 2030 are used to provide 2019 and 2020 estimates. 


A script was created to address the data request, which allows standard Soundcast network outputs to be segmented to the jurisdiction level. The "aq_tool" notebook processes Soundcast output for a list of provided cities and reports emissions and VMT by light, medium, and heavy vehicles. 

This notebook compiles and organizes the results of the aq_tool.

In [665]:
import os
import pandas as pd

In [666]:
# Location of output from aq_tool
output_dir = r'C:\Workspace\aq_tool\output'

In [667]:
year = '2018'
df_18 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2030'
df_30 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2040'
df_40 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2050'
df_50 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

In [668]:
# Create interpolations
df = df_18.merge(df_30, on=['pollutant_name','city','veh_type'], suffixes=['_2018','_2030'], how='outer')

# Merge 2040
year = '2040'
df = df.merge(df_40, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)

# Merge 2050
year = '2050'
df = df.merge(df_50, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)


# df.head()

In [669]:
def interpolate(df, start_year, end_year, interpolated_year, var):
    """Interpolate values from two cols of data given start, end, and target (interpolated) years."""
    
    # Calculate annual rate of change based on difference between values at start and end year, divided number of years difference
    df['annual_'+var+'_change'] = (df[var+'_'+str(end_year)] - df[var+'_'+str(start_year)])/(end_year-start_year)
    # Calculate 
    df[var+'_'+str(interpolated_year)] = df[var+'_'+str(start_year)]+df['annual_'+var+'_change']*(interpolated_year-start_year)

    return df                                 

In [670]:
df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')


In [671]:
df_vmt = df_vmt[['city','pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020','vmt_2030','vmt_2040','vmt_2050']]

# VMT is duplicated for each pollutant total; Select only the first set of rows
df_vmt = df_vmt.drop('pollutant_name', axis=1)
df_vmt = df_vmt.groupby(['city','veh_type']).first()

df_vmt.to_csv(r'C:\Workspace\aq_tool\output\king_county_cities_vmt.csv')

In [672]:
# Get county totals by vehicle type
df_city_tot_vmt = df_vmt.groupby('veh_type').sum().reset_index()

#### Compile Emissions

In [673]:
df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')

In [674]:
df_emissions = df_emissions[['city','pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
    'total_daily_tons_2020','total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]
df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent'])]

In [675]:
df_emissions.to_csv(r'C:\Workspace\aq_tool\output\king_county_cities_emissions.csv', index=False)

In [676]:
# Get county totals by vehicle type
df_city_tot_emissions = df_emissions.drop('city', axis=1).groupby(['pollutant_name','veh_type']).sum().reset_index()

### Compile Totals for County and Unincorporated Areas

Get county level results from the outputs of the regional results

In [687]:
def get_county_emissions(run_dir, county=None):
    
    # Get county level totals
    df_interzonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\interzonal_emissions.csv'))
    df_intrazonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\intrazonal_emissions.csv'))
    start_emissions_df = pd.read_csv(os.path.join(r'outputs\emissions\start_emissions.csv'))
    df_inter_group = df_interzonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_inter_group.rename(columns={'tons_tot': 'interzonal_tons'}, inplace=True)
    df_intra_group = df_intrazonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_intra_group.rename(columns={'tons_tot': 'intrazonal_tons'}, inplace=True)
    df_start_group = start_emissions_df.groupby(['pollutantID','veh_type','county']).sum()[['start_tons']].reset_index()

    from emissions import finalize_emissions, pollutant_map
    summary_df = pd.merge(df_inter_group, df_intra_group, on=['veh_type','pollutantID','county'], suffixes=['_interzonal','_intrazonal'])
    summary_df['vmt'] = summary_df['vmt_interzonal']+summary_df['vmt_intrazonal']
    summary_df = pd.merge(summary_df, df_start_group, how='left', on=['veh_type','pollutantID','county'])
    # summary_df = finalize_emissions(summary_df, col_suffix="")
    pm10 = summary_df[summary_df['pollutantID'].isin([100,106,107])].groupby(['veh_type','county']).sum().reset_index()
    pm10['pollutantID'] = 'PM10'
    pm25 = summary_df[summary_df['pollutantID'].isin([110,116,117])].groupby(['veh_type','county']).sum().reset_index()
    pm25['pollutantID'] = 'PM25'
    summary_df = summary_df.append(pm10)
    summary_df = summary_df.append(pm25)
    summary_df.loc[~summary_df['pollutantID'].isin(['PM','PM10','PM25']),'pollutantID'] = summary_df[~summary_df['pollutantID'].isin(['PM','PM10','PM25'])]['pollutantID'].astype('int')
    summary_df['pollutant_name'] = summary_df['pollutantID'].astype('int', errors='ignore').astype('str').map(pollutant_map)
    summary_df['total_daily_tons'] = summary_df['start_tons']+summary_df['interzonal_tons']+summary_df['intrazonal_tons']
    summary_df = summary_df[['pollutantID','county','pollutant_name','veh_type','start_tons','intrazonal_tons','interzonal_tons','total_daily_tons','vmt']]

    # Filter for county
    if county:
        summary_df = summary_df[summary_df['county'] == county]

    return summary_df

In [688]:
run_dir = r'\\modelstation2\c$\Workspace\sc_2018_rtp_final\soundcast'
df_18_county = get_county_emissions(run_dir, 'king')

run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2030_final\soundcast'
df_30_county = get_county_emissions(run_dir, 'king')

run_dir = r'\\modelstation1\c$\workspace\sc_2040_rtp_final\soundcast'
df_40_county = get_county_emissions(run_dir, 'king')

run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2050_constrained_final\soundcast'
df_50_county = get_county_emissions(run_dir, 'king')


# Create interpolations
df = df_18_county.merge(df_30_county, on=['pollutant_name','veh_type'], suffixes=['_2018','_2030'], how='outer')

# Merge 2040
year = '2040'
df = df.merge(df_40_county, on=['pollutant_name','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)

# # Merge 2050
year = '2050'
df = df.merge(df_50_county, on=['pollutant_name','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)


df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
df_vmt = df_vmt[['pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020','vmt_2030','vmt_2040','vmt_2050']]

df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
df_emissions = df_emissions[['pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
    'total_daily_tons_2020','total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]


In [679]:
df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent'])]
df_emissions.to_csv(r'C:\Workspace\aq_tool\output\king_county_total_emissions.csv', index=False)



In [721]:
_df_vmt = df_vmt.groupby('veh_type').first().reset_index()
_df_vmt['geography'] = 'King County Total'
_df_vmt.drop('pollutant_name', axis=1, inplace=True)
_df_vmt.to_csv(r'C:\Workspace\aq_tool\output\king_county_total_vmt.csv', index=False)

In [692]:
# 
df = df_city_tot_emissions.merge(df_emissions, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2030,2040,2050]:
    df['total_daily_tons_unincorporated_'+str(year)] = df['total_daily_tons_'+str(year)+'_county_tot']-df['total_daily_tons_'+str(year)+'_cities_tot']

In [693]:
df[['pollutant_name','veh_type']+['total_daily_tons_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2030','2040','2050']]].to_csv(r'C:\Workspace\aq_tool\output\king_county_unincorporated_emissions.csv', index=False)

In [715]:
df = _df_vmt.drop('geography', axis=1).merge(df_city_tot_vmt, suffixes=['_county','_city_total'], on='veh_type')
# df = df_vmt.merge(df_emissions, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2030,2040,2050]:
    df['vmt_unincorporated_'+str(year)] = df['vmt_'+str(year)+'_county']-df['vmt_'+str(year)+'_city_total']

In [718]:
df['geography'] = 'Unincorporated King'
df[['veh_type','geography']+['vmt_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2030','2040','2050']]].to_csv(r'C:\Workspace\aq_tool\output\king_county_unincorporated_vmt.csv', index=False)